In [185]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-0aea  GCE       2                                             RUNNING  us-central1-a


# Imports & Setup

In [187]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [188]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
import hashlib
from nltk.util import ngrams
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *
from nltk.stem import WordNetLemmatizer
from math import log10
from inverted_index_gcp import InvertedIndex

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [189]:
spark

In [190]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'tfidf_bucket_318437159' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh':
        paths.append(full_path+b.name)

# Getting the data

In [191]:
# adding our python module to the cluster
%cd -q /home/dataproc
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

24/03/02 19:20:25 WARN SparkContext: The path /home/dataproc/inverted_index_gcp.py has been added already. Overwriting of added paths is not supported in the current version.


# Building an inverted index

In [192]:
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

SIZE_OF_WIKI = 6348910
LONGEST_WIKI_ARTICLE = 17659

stemmer = PorterStemmer()
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became",  "make",
                    "accordingly", "hence",
    "namely",
    "therefore",
    "thus",
    "consequently",
    "meanwhile",
    "accordingly",
    "likewise",
    "similarly",
    "notwithstanding",
    "nonetheless",
    "despite",
    "whereas",
    "furthermore",
    "moreover",
    "nevertheless",
    "although",
    "notably",
    "notwithstanding",
    "nonetheless",
    "despite",
    "whereas",
    "furthermore",
    "moreover",
    "notably", "hence"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
    return int(_hash(token),16) % NUM_BUCKETS


def word_count(text, id, biagram, use_doc_len):
    ''' Count the frequency of each word in `text` (tf) that is not included in
    `all_stopwords` and return entries that will go into our posting lists.
    Parameters:
    -----------
    text: str
      Text of one document
    id: int
      Document id
    Returns:
    --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs
      for example: [("Anarchism", (12, 5)), ...]
    '''
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    tokens = [tok for tok in tokens if tok not in all_stopwords]

    tokens = [stemmer.stem(token) for token in tokens]
    if biagram:
        tokens = list(ngrams(tokens, 2))
        tokens = [' '.join(bigram) for bigram in tokens]
    counter = Counter(tokens)
    normalizer = counter.most_common(1)[0][1] if counter else 1
    if use_doc_len:
        normalizer = len(tokens) 
    return [(token, (id, count/normalizer)) for token, count in counter.items()]

def reduce_word_counts(unsorted_pl):
    ''' Returns a sorted posting list by wiki_id.
    Parameters:
    -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples
    Returns:
    --------
    list of tuples
      A sorted posting list.
    '''
    return sorted(unsorted_pl, key=lambda x: x[0])

# def add_tfIDF(postingList):
#     word_freq = len(postingList)
#     newPost = []
#     for post in postingList:
#         newPost.append((post[0], post[1]*log10(SIZE_OF_WIKI / word_freq)))
#     return newPost

def calculate_df(postings):
    ''' Takes a posting list RDD and calculate the df for each token.
    Parameters:
    -----------
      postings: RDD
        An RDD where each element is a (token, posting_list) pair.
    Returns:
    --------
      RDD
        An RDD where each element is a (token, df) pair.
    '''
    # Count the document frequency for each token
    return postings.map(lambda x: (x[0], len(x[1])))

def partition_postings_and_write(postings, folder_name):
    ''' A function that partitions the posting lists into buckets, writes out
    all posting lists in a bucket to disk, and returns the posting locations for
    each bucket. Partitioning should be done through the use of `token2bucket`
    above. Writing to disk should use the function  `write_a_posting_list`, a
    static method implemented in inverted_index_colab.py under the InvertedIndex
    class.
    Parameters:
    -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
    Returns:
    --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and
      offsets its posting list was written to. See `write_a_posting_list` for
      more details.
    '''
    # posting in postings is word and posting list tuple
    bucketed_postings = postings.map(lambda x: (token2bucket_id(x[0]), x))
    # Group postings by bucket ID
    grouped_postings = bucketed_postings.groupByKey()
    # Write each bucket's postings to disk and collect locations
    posting_locations = grouped_postings.map(lambda x: InvertedIndex.write_a_posting_list(x, folder_name, bucket_name))

    return posting_locations

In [193]:
def create_posting_locs(doc_text_pairs, folder_name, stemOrBiagrm, doc_len, filter_thershold):
    # word counts map
    word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1], stemOrBiagrm, doc_len))
    postings = word_counts.groupByKey().mapValues(reduce_word_counts)
    postings = postings.mapValues(add_tfIDF)# filtering postings and calculate df
    postings_filtered = postings.filter(lambda x: len(x[1]) > filter_thershold)
    w2df = calculate_df(postings_filtered)
    w2df_dict = w2df.collectAsMap()
    # partition posting lists and write out
    partition_postings_and_write(postings_filtered, folder_name).collect()
    return w2df_dict
    

In [194]:
def read_posting_locs(folder_name):
    # collect all posting lists locations into one super-set
    super_posting_locs = defaultdict(list)
    for blob in client.list_blobs(bucket_name, prefix=f'{folder_name}'):
        if not blob.name.endswith("pickle"):
            continue
        with blob.open("rb") as f:
            posting_locs = pickle.load(f)
            for k, v in posting_locs.items():
                super_posting_locs[k].extend(v)
    return super_posting_locs

Putting it all together

In [195]:
def create_posting_map(super_posting_locs, w2df_dict, inverted_name):    
    # Create inverted index instance
    inverted = InvertedIndex()
    # Adding the posting locations dictionary to the inverted index
    inverted.posting_locs = super_posting_locs
    # Add the token - df dictionary to the inverted index
    inverted.df = w2df_dict
    # write the global stats out
    inverted.write_index('.', inverted_name)
    # upload to gs
    index_src = "index.pkl"
    index_dst = f'gs://{bucket_name}/inverted_indices/{index_src}'
    !gsutil cp $index_src $index_dst
    return inverted

In [196]:
def create_invertedIndex(doc_text_pairs, folder_name, Biagram, doc_len, filter_thershold):
    w2df_dict = create_posting_locs(doc_text_pairs, folder_name, Biagram, doc_len, filter_thershold)
    super_posting_locs = read_posting_locs(folder_name)
    return create_posting_map(super_posting_locs, w2df_dict, folder_name)


In [ ]:
STEMMING_BODY_FOLDER = 'inverted_test_text_with_stemming'
BIGRAM_BODY_FOLDER = 'inverted_test_text_with_bigram'
STEMMING_TITLE_FOLDER = 'inverted_test_title_with_stemming'
BIGRAM_TITLE_FOLDER = 'inverted_test_title_with_bigram'
STEMMING_BODY_MOST_COMMON_FOLDER = 'inverted_test_body_most_common_with_stemming'
STEMMING_TITLE_MOST_COMMON_FOLDER = 'inverted_test_title_most_common_with_stemming'

parquetFile = spark.read.parquet(*paths)
text_data = parquetFile.limit(10000).select("text", "id").rdd  
title_data = parquetFile.limit(10000).select("title", "id").rdd  
inverted1 = create_invertedIndex(text_data, f'{STEMMING_BODY_FOLDER}', False, True, 10)
inverted3 = create_invertedIndex(title_data, f'{STEMMING_TITLE_FOLDER}', False , True, 4)
text_data = parquetFile.limit(10000).select("text", "id").rdd  
title_data = parquetFile.limit(10000).select("title", "id").rdd  
inverted2 = create_invertedIndex(text_data, f'{BIGRAM_BODY_FOLDER}', True , True, 10)
inverted4 = create_invertedIndex(title_data, f'{BIGRAM_TITLE_FOLDER}', True , True, 4)
text_data = parquetFile.limit(10000).select("text", "id").rdd  
title_data = parquetFile.limit(10000).select("title", "id").rdd  
inverted5 = create_invertedIndex(text_data, f'{BIGRAM_BODY_MOST_COMMON_FOLDER}', True , False, 10)
inverted6 = create_invertedIndex(title_data, f'{BIGRAM_TITLE_MOST_COMMON_FOLDER}', True, False, 4)

# PageRank

In [173]:
print(inverted6.df)


{'high school': 79}


In [78]:
def generate_graph(pages):
    vertices = pages.flatMap(lambda page: [(page[0], )] + [(link[0], ) for link in page[1]])

    edges = pages.flatMap(lambda page: [(page[0], link[0]) for link in page[1]])

    edges = edges.distinct()
    vertices = vertices.distinct()

    return edges, vertices

In [79]:
t_start = time()
pages_links = parquetFile.limit(10000).select("id","anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
pr_time = time() - t_start
pr.show()

/usr/lib/spark/python/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/usr/lib/spark/python/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


+-------+------------------+
|     id|          pagerank|
+-------+------------------+
|  68253| 404.5411250314113|
|  41512| 403.9501793282662|
|1234326|  403.837684837305|
| 382207|403.32764088914206|
|  53509|393.25732447614223|
| 345173| 246.6592577008005|
|  44812|206.22810908594528|
| 344869|  102.311634129927|
|  74594|  65.2200954494868|
|  20593|56.835850990358374|
|  16815| 51.92827670733025|
|6769081|30.796224517795995|
|6650649| 29.07635029356804|
|6613141|28.476027061310425|
| 386690| 25.34798375347078|
| 386720| 24.06624301563353|
|7036474|  22.3572316936504|
| 386723|21.977433179754335|
|6662872| 20.88566477845661|
| 171166|20.270344350725992|
+-------+------------------+
only showing top 20 rows



In [184]:
def filter_stop_words(text):

    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    tokens = [tok for tok in tokens if tok not in all_stopwords]
    tokens = [stemmer.stem(token) for token in tokens]
    tokens = list(ngrams(tokens, 2))
    return [' '.join(bigram) for bigram in tokens]


title_data = parquetFile.limit(10000).select("title", "id").rdd
word_counts = title_data.flatMap(lambda text: filter_stop_words(text[0]))

print(word_counts.count())
               

15013
